In [1]:
import torch
from torch.utils.data import DataLoader
from dataset.dataset import get_cdiscount_dataset
from model.model import assemble_model
from trainer.trainer import get_trainer

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# redirect print to file
# import sys
# sys.stdout = open("PyTorch-resnet34-log.txt", "w")

In [2]:
# parameters
config = {
    'train_batch_size': 256, 'val_batch_size': 256,
    'arch': 'resnet34',
    'optimizer': 'Adam', 'learning_rate': 1e-3, 'decay_lr_freq': 1e3, 'weight_decay': 5e-4,
    'resume': None,
    'start_epoch': 0, 'epochs': 10,
    'print_freq': 10, 'validate_freq': 3e4, 'save_freq': 1e3,
    'best_val_prec1': 0
}

In [3]:
import torchvision.models as models

# get dataset
print('getting dataset...')
train_dataset = get_cdiscount_dataset(offsets_csv="train_offsets.csv",
                                      images_csv="train_images.csv",
                                      bson_file_path="/mnt/data/cdiscount/train.bson",
                                      with_label=True,
                                      resize=256)
val_dataset = get_cdiscount_dataset(offsets_csv="train_offsets.csv",
                                    images_csv="val_images.csv",
                                    bson_file_path="/mnt/data/cdiscount/train.bson",
                                    with_label=True,
                                    resize=256)

# get data loader
print('getting data loader...')
train_dataloader = DataLoader(train_dataset, batch_size=config['train_batch_size'], shuffle=True, num_workers=6)
val_dataloader = DataLoader(val_dataset, batch_size=config['val_batch_size'], shuffle=True, num_workers=6)

# define model
print("=> using pre-trained model '{}'".format(config['arch']))
model = models.__dict__[config['arch']](pretrained=True)
model = assemble_model(model, -1, 512, 5270)
model = torch.nn.DataParallel(model).cuda()

# define loss function (criterion) and optimizer
criterion = torch.nn.CrossEntropyLoss().cuda()

# get trainer
Trainer = get_trainer(train_dataloader, val_dataloader, model, criterion, config)

# Run!
Trainer.run()

getting dataset...
getting data loader...
=> using pre-trained model 'resnet34'
start training
Epoch: [0][0/38676]	Time 25.404 (25.404)	Data 9.500 (9.500)	Loss 8.8635 (8.8635)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
adjust learning rate, decay_rate = 1
adjust learning rate, decay_rate = 2
Epoch: [0][10/38676]	Time 3.804 (5.674)	Data 0.006 (0.872)	Loss 7.5447 (7.6427)	Prec@1 6.250 (5.043)	Prec@5 11.328 (9.837)
adjust learning rate, decay_rate = 3
adjust learning rate, decay_rate = 4


Process Process-3:
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 40, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
Process Process-1:
Process Process-6:
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-4:
Process Process-5:
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 34, in _worker_loop
    r = index_queue.get()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/mult

KeyboardInterrupt: 